# DATASHEET DRILL — IRFZ44N Datasheet Walkthrough

**Module 04 — MOSFETs | Notebook 03**

---

The IRFZ44N is a workhorse N-channel power MOSFET. It's been in production for decades and shows up in countless hobby and professional projects. We're going to walk through its datasheet section by section, understanding every key parameter.

**What you'll learn:** How to read any MOSFET datasheet. The parameters are the same across all devices — once you understand the IRFZ44N datasheet, you can read any MOSFET datasheet.

Pull up the IRFZ44N datasheet (from International Rectifier / Infineon) and follow along.

## Part Overview

**IRFZ44N** — N-Channel Power MOSFET

| Feature | Value |
|---|---|
| Type | N-channel enhancement mode |
| Package | TO-220AB |
| V_DSS (max drain-source voltage) | 55 V |
| I_D (continuous drain current) | 49 A (at T_C = 25°C) |
| R_DS(on) | 17.5 mΩ typical |
| Q_g (total gate charge) | 63 nC typical |

### TO-220 Package

```
    Front view (text facing you, leads pointing down):
    
    ┌─────────────┐
    │             │ ← Metal tab (= DRAIN)
    │   IRFZ44N   │
    │             │
    └──┬──┬──┬──┘
       │  │  │
       G  D  S
      (1)(2)(3)
    
    Pin 1: Gate
    Pin 2: Drain (also the metal tab!)
    Pin 3: Source
```

**Important:** The metal tab is electrically connected to the **Drain**. If you bolt this to a heatsink, the heatsink will be at drain potential! Use an insulating pad if the heatsink is grounded.

## Absolute Maximum Ratings

These are the **never-exceed** limits. Exceeding any of these, even briefly, may destroy the device.

| Parameter | Symbol | Value | Notes |
|---|---|---|---|
| Drain-Source Voltage | V_DSS | 55 V | Max voltage across drain-source |
| Gate-Source Voltage | V_GS | ±20 V | Exceeding this destroys the gate oxide! |
| Continuous Drain Current | I_D | 49 A (T_C=25°C) | Package thermal limit, not silicon limit |
| Continuous Drain Current | I_D | 36 A (T_C=100°C) | Derated because package can't dissipate as much heat |
| Pulsed Drain Current | I_DM | 160 A | For very short pulses only |
| Power Dissipation | P_D | 94 W (T_C=25°C) | With infinite heatsink (case at 25°C) |
| Junction Temperature | T_J | 175 °C | Absolute max silicon temperature |

### What these really mean:

- **V_DSS = 55V:** Your drain-source voltage must never exceed 55V, including transient spikes. In a 12V system, you have plenty of margin. In a 48V system, you'd need to worry about inductive spikes.

- **V_GS = ±20V:** The gate oxide is only nanometers thick. At 20V, the electric field across it approaches breakdown. This is why ESD protection matters!

- **I_D = 49A:** This isn't a silicon limit — the silicon could handle more. It's the **package thermal limit**. The TO-220 package can only conduct so much heat away from the die. At 100°C case temperature, it drops to 36A.

- **P_D = 94W:** Only achievable with the case held at 25°C (requires a very good heatsink). In free air, you'll get maybe 2-3W before overheating.

## Key Electrical Parameters

### V_GS(th) — Gate Threshold Voltage

| Parameter | Min | Typ | Max | Test Conditions |
|---|---|---|---|---|
| V_GS(th) | 2.0 V | — | 4.0 V | V_DS = V_GS, I_D = 250 μA |

**What this means:**
- The MOSFET starts conducting somewhere between 2.0V and 4.0V on the gate
- The "typical" value isn't specified — it varies widely between units
- Test condition: V_DS = V_GS means drain and gate are shorted. I_D = 250 μA defines "starts conducting" — this is a trickle, not useful current
- **The device is NOT fully on at V_GS(th)!** You need much more gate voltage for low R_DS(on)

### Temperature dependence:
V_GS(th) decreases with temperature (about -5 to -7 mV/°C). A device with V_GS(th) = 3V at 25°C might have V_GS(th) = 2.3V at 125°C. This is important for worst-case analysis.

### R_DS(on) — On-State Drain-Source Resistance

| Parameter | Typ | Max | Test Conditions |
|---|---|---|---|
| R_DS(on) | 17.5 mΩ | 24 mΩ | V_GS = 10V, I_D = 25A |

**What this means:**
- When fully on with V_GS = 10V, the MOSFET looks like a 17.5 mΩ resistor (typical)
- Worst case (guaranteed maximum): 24 mΩ
- **Always design for the max value**, not typical!
- The test conditions are critical: R_DS(on) is specified at V_GS = 10V and I_D = 25A

**Why test conditions matter:**
- At V_GS = 5V, R_DS(on) might be 30-50 mΩ — much worse!
- At V_GS = 4.5V (typical logic level), it might be 60-100 mΩ
- At T_J = 125°C, R_DS(on) increases to roughly 2× the 25°C value

This is why the IRFZ44N is **NOT a logic-level MOSFET** — you need 10V gate drive for the specified R_DS(on).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Recreate R_DS(on) vs V_GS characteristic from IRFZ44N datasheet
# This shows why gate drive voltage matters enormously

# Approximate data from datasheet (R_DS(on) vs V_GS at I_D = 25A, T_J = 25°C)
VGS_data = np.array([4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8.0, 10.0, 12.0, 15.0, 20.0])
RDS_data = np.array([200, 80, 45, 32, 26, 21, 19, 17.5, 16.5, 16.0, 15.5])  # mΩ

# Interpolate for smooth curve
VGS_fine = np.linspace(4.0, 20.0, 300)
RDS_fine = np.interp(VGS_fine, VGS_data, RDS_data)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: R_DS(on) vs V_GS
ax1.plot(VGS_fine, RDS_fine, 'b-', linewidth=2.5)
ax1.plot(VGS_data, RDS_data, 'bo', markersize=6)

# Mark key operating points
key_points = {
    '3.3V logic': (4.0, 200),
    '5V logic': (5.0, 45),
    '10V (datasheet spec)': (10.0, 17.5),
    '12V (gate driver)': (12.0, 16.5),
}

for label, (vgs, rds) in key_points.items():
    ax1.annotate(f'{label}\n{rds} mΩ', xy=(vgs, rds),
               xytext=(vgs + 1.5, rds + 30),
               fontsize=9, arrowprops=dict(arrowstyle='->', color='red'),
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

ax1.set_xlabel('V_GS (V)', fontsize=12)
ax1.set_ylabel('R_DS(on) (mΩ)', fontsize=12)
ax1.set_title('IRFZ44N: R_DS(on) vs V_GS\n(I_D = 25A, T_J = 25°C)', fontsize=13)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(3, 21)
ax1.set_ylim(0, 250)

# Right: R_DS(on) vs Temperature
T = np.linspace(25, 175, 200)
# IRFZ44N R_DS(on) roughly doubles from 25°C to 150°C
# Normalized to 1.0 at 25°C
RDS_norm = 1.0 + 0.0065 * (T - 25)  # ~+0.65%/°C
RDS_vs_T = 17.5 * RDS_norm  # Absolute values in mΩ

ax2.plot(T, RDS_vs_T, 'r-', linewidth=2.5)
ax2.axhline(y=17.5, color='blue', linestyle='--', alpha=0.5, label='25°C spec (17.5 mΩ)')
ax2.axhline(y=24, color='orange', linestyle='--', alpha=0.5, label='Datasheet max (24 mΩ)')

# Mark some key temperatures
for temp in [25, 75, 125, 175]:
    rds_at_temp = 17.5 * (1.0 + 0.0065 * (temp - 25))
    ax2.plot(temp, rds_at_temp, 'ro', markersize=6)
    ax2.annotate(f'{rds_at_temp:.1f} mΩ', xy=(temp, rds_at_temp),
               xytext=(temp + 5, rds_at_temp + 2), fontsize=9)

ax2.set_xlabel('Junction Temperature (°C)', fontsize=12)
ax2.set_ylabel('R_DS(on) (mΩ)', fontsize=12)
ax2.set_title('IRFZ44N: R_DS(on) vs Temperature\n(V_GS = 10V)', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(20, 180)
ax2.set_ylim(0, 40)

plt.tight_layout()
plt.show()

print("LEFT: R_DS(on) drops dramatically as you increase V_GS — then flattens out.")
print("      At 5V gate, R_DS(on) is 2.6x worse than at 10V!")
print("      Beyond ~10V, there's diminishing return.")
print("")
print("RIGHT: R_DS(on) nearly doubles from 25°C to 175°C.")
print("       ALWAYS use the hot R_DS(on) for thermal calculations!")

### I_D — Continuous Drain Current

The datasheet says 49A at T_C = 25°C. But what does this really mean?

This is calculated from: **I_D = √(P_D / R_DS(on))**

Where P_D is limited by the **thermal resistance from junction to case** and the maximum junction temperature:

- P_D(max) = (T_J(max) - T_C) / R_θJC = (175°C - 25°C) / 1.6°C/W = 94W
- I_D = √(94W / 0.0175Ω) = √5371 = 73A ← but limited to 49A by bond wires/package

**Reality check:** You'll never run 49A through a TO-220 on a breadboard. The package would melt. This rating assumes the case is held at 25°C by a large heatsink. In practice:
- Free air (no heatsink): ~3-5A max
- Small heatsink: ~10-15A
- Large heatsink with fan: ~20-30A

### Q_g — Total Gate Charge

| Parameter | Typ | Max | Test Conditions |
|---|---|---|---|
| Q_g (total) | 63 nC | 80 nC | V_GS = 10V, V_DS = 44V, I_D = 25A |
| Q_gs | 15 nC | — | |
| Q_gd | 28 nC | — | |

**Why gate charge matters:**

1. **Switching speed:** To switch in time t, the gate driver must supply current I_g = Q_g / t.
   - To switch in 100 ns: I_g = 63 nC / 100 ns = 630 mA of gate drive current!
   - To switch in 1 μs: I_g = 63 nC / 1 μs = 63 mA

2. **Gate drive power:** P_gate = Q_g × V_GS × f_sw
   - At 100 kHz: P = 63 nC × 10V × 100 kHz = 63 mW
   - At 1 MHz: P = 63 nC × 10V × 1 MHz = 630 mW — significant!

3. **Driver selection:** Your gate driver must be able to source/sink the required peak current.

Q_gd is particularly important — this is the **Miller charge** that must be supplied during the Miller plateau, which is the slowest part of the switching transition.

### Switching Times

| Parameter | Typ | Test Conditions |
|---|---|---|
| t_d(on) — turn-on delay | 12 ns | V_DD = 27V, I_D = 25A |
| t_r — rise time | 60 ns | R_G = 4.3Ω |
| t_d(off) — turn-off delay | 30 ns | |
| t_f — fall time | 30 ns | |

**Total switching time:**
- Turn-on: t_d(on) + t_r = 12 + 60 = 72 ns
- Turn-off: t_d(off) + t_f = 30 + 30 = 60 ns

**Important:** These times depend on the gate drive circuit (R_G = 4.3Ω in the test). With a weaker driver (higher R_G), switching will be slower. With a stronger driver (lower R_G), it can be faster.

### Body Diode Parameters

| Parameter | Value | Notes |
|---|---|---|
| I_S (continuous body diode current) | 49 A | Same as I_D |
| V_SD (body diode forward voltage) | 1.3 V typ | At I_S = 25A |
| t_rr (reverse recovery time) | 51 ns typ | Slow compared to fast diodes |

The body diode's V_SD of 1.3V is quite high — an external Schottky diode (0.3-0.5V) in parallel can reduce losses in applications where the body diode conducts (e.g., motor drives).

## Characteristic Curves — What the Datasheet Graphs Tell You

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Recreate IRFZ44N output characteristics (I_D vs V_DS)
# Using approximate values from the datasheet

VDS = np.linspace(0, 10, 500)
V_th = 3.0  # approximate for this device
K = 15.0    # adjusted for power MOSFET (much higher than small-signal)

VGS_values = [4.0, 4.5, 5.0, 6.0, 7.0, 8.0, 10.0]
colors = plt.cm.plasma(np.linspace(0.1, 0.9, len(VGS_values)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Output characteristics
for VGS, color in zip(VGS_values, colors):
    V_ov = VGS - V_th
    if V_ov <= 0:
        continue
    V_DS_sat = V_ov
    
    ID = np.where(
        VDS < V_DS_sat,
        K * (2 * V_ov * VDS - VDS**2),
        K * V_ov**2 * (1 + 0.01 * (VDS - V_DS_sat))
    )
    ID = np.clip(ID, 0, 80)  # limit for visibility
    
    ax1.plot(VDS, ID, color=color, linewidth=2, label=f'V_GS = {VGS}V')

ax1.set_xlabel('V_DS (V)', fontsize=12)
ax1.set_ylabel('I_D (A)', fontsize=12)
ax1.set_title('Output Characteristics: I_D vs V_DS', fontsize=13)
ax1.legend(fontsize=9, loc='right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 5)
ax1.set_ylim(0, 80)

# Annotate the ohmic region
ax1.annotate('Ohmic (switch) region\nV_DS small, behaves as R_DS(on)',
            xy=(0.3, 40), fontsize=9,
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

# Transfer characteristics (I_D vs V_GS at V_DS = 10V)
VGS_transfer = np.linspace(0, 10, 500)
ID_transfer = np.where(VGS_transfer > V_th,
                       K * (VGS_transfer - V_th)**2,
                       0)
ID_transfer = np.clip(ID_transfer, 0, 100)

ax2.plot(VGS_transfer, ID_transfer, 'b-', linewidth=2.5)
ax2.axvline(x=V_th, color='red', linestyle='--', alpha=0.7, label=f'V_GS(th) ≈ {V_th}V')

# Mark useful operating points
for vgs_mark in [4.5, 5.0, 7.0, 10.0]:
    id_mark = min(K * (vgs_mark - V_th)**2, 100)
    ax2.plot(vgs_mark, id_mark, 'ro', markersize=6)
    ax2.annotate(f'{vgs_mark}V → {id_mark:.0f}A', xy=(vgs_mark, id_mark),
               xytext=(vgs_mark + 0.3, id_mark + 3), fontsize=9)

ax2.set_xlabel('V_GS (V)', fontsize=12)
ax2.set_ylabel('I_D (A)', fontsize=12)
ax2.set_title('Transfer Characteristic: I_D vs V_GS\n(V_DS = 10V, T_J = 25°C)', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

print("LEFT: Family of curves. In the ohmic region (small V_DS), the curves are steep")
print("      and linear — the MOSFET acts like a resistor (R_DS(on)).")
print("")
print("RIGHT: The square-law relationship is visible. Current rises rapidly with V_GS.")
print("       Notice how much more current flows at V_GS = 10V vs 5V!")

## The Material Science Why

### Why R_DS(on) depends on V_GS

The channel resistance has two components:

1. **Channel resistance** — The thin inversion layer under the gate. More V_GS = stronger field = more electrons = lower channel resistance. This dominates at low V_GS.

2. **Drift region resistance** — The lightly-doped N- region between the channel and the drain. This is voltage-independent and sets the floor for R_DS(on). This is why there's diminishing return above ~10V gate drive.

For a 55V MOSFET like the IRFZ44N, the drift region must be thick and lightly doped to support 55V without breakdown. This drift region contributes most of the R_DS(on). **Higher voltage rating = thicker drift region = higher R_DS(on)** — this is a fundamental tradeoff.

### Why the IRFZ44N is a "trench" MOSFET

Modern power MOSFETs like the IRFZ44N use a **trench gate** structure: the gate electrode is etched into a trench in the silicon rather than sitting on top. This allows:
- Smaller cell pitch (more cells in parallel per chip area)
- Better channel conductivity
- Lower R_DS(on) for a given chip size

## Safe Operating Area (SOA)

The SOA graph shows the simultaneous V_DS and I_D limits. You must stay inside all boundaries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# IRFZ44N Safe Operating Area (SOA) — approximate reconstruction

fig, ax = plt.subplots(figsize=(10, 7))

# SOA boundaries (log-log plot)
# 1. Current limit (horizontal line at I_D max)
# 2. R_DS(on) limit (diagonal line: I_D = V_DS / R_DS(on))
# 3. Power limit (hyperbola: I_D = P_max / V_DS)
# 4. Voltage limit (vertical line at V_DS max)

V_DS = np.logspace(-1, 2, 1000)  # 0.1 to 100V

# Boundaries
I_D_max = 49  # A (current limit)
R_DS_on = 0.0175  # Ω
P_max_DC = 94  # W (DC power limit at T_C = 25°C)
V_DS_max = 55  # V

# R_DS(on) limit: I_D = V_DS / R_DS(on)
I_rds = V_DS / R_DS_on

# Power limits for different pulse widths
pulse_data = [
    ('DC', P_max_DC, 'blue'),
    ('1 ms', P_max_DC * 5, 'green'),
    ('100 μs', P_max_DC * 20, 'orange'),
    ('10 μs', P_max_DC * 100, 'red'),
]

for label, p_lim, color in pulse_data:
    I_power = p_lim / V_DS
    
    # SOA boundary is the minimum of all limits
    I_soa = np.minimum(I_D_max, np.minimum(I_rds, I_power))
    # Apply voltage limit
    I_soa = np.where(V_DS <= V_DS_max, I_soa, 0.001)
    
    ax.loglog(V_DS, I_soa, linewidth=2, color=color, label=f'{label} pulse')

# Fill the DC SOA region
I_dc_soa = np.minimum(I_D_max, np.minimum(I_rds, P_max_DC / V_DS))
I_dc_soa = np.where(V_DS <= V_DS_max, I_dc_soa, 0.001)
ax.fill_between(V_DS, 0.01, I_dc_soa, alpha=0.1, color='blue',
               where=(V_DS <= V_DS_max))

# Labels for each boundary
ax.annotate('I_D limit\n(49A)', xy=(0.3, 49), fontsize=10, color='black',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.annotate('R_DS(on) limit', xy=(0.15, 8), fontsize=10, color='black', rotation=45,
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.annotate('Power\nlimit (DC)', xy=(10, 8), fontsize=10, color='blue',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.annotate('V_DS limit\n(55V)', xy=(55, 5), fontsize=10, color='black',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.set_xlabel('V_DS (V)', fontsize=12)
ax.set_ylabel('I_D (A)', fontsize=12)
ax.set_title('IRFZ44N Safe Operating Area (SOA)\nT_C = 25°C', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')
ax.set_xlim(0.1, 100)
ax.set_ylim(0.01, 500)

plt.tight_layout()
plt.show()

print("The SOA defines the SAFE operating region. Stay INSIDE the boundary!")
print("")
print("Four boundaries:")
print("  1. Horizontal ceiling: max I_D (49A) — bond wire/package limit")
print("  2. Diagonal (upper-left): R_DS(on) limit — can't have V_DS < I_D × R_DS(on)")
print("  3. Diagonal (center): power limit — P = V_DS × I_D ≤ P_max")
print("  4. Vertical wall: V_DS max (55V) — breakdown voltage")
print("")
print("For pulsed operation (short duty cycle), the power limit moves outward.")

## Thermal Design — The Most Important Calculation

The IRFZ44N's thermal resistance chain:

```
    Junction (T_J)  →  Case (T_C)  →  Heatsink (T_S)  →  Ambient (T_A)
              R_θJC = 1.6°C/W   R_θCS ≈ 0.5°C/W   R_θSA = depends on heatsink
```

**The fundamental equation:**

T_J = T_A + P_D × (R_θJC + R_θCS + R_θSA)

Where:
- T_J = junction temperature (must stay below 175°C, design target < 125°C)
- T_A = ambient temperature (typically 25-50°C)
- P_D = power dissipation = I_D² × R_DS(on)
- R_θJC = junction-to-case thermal resistance = **1.6°C/W** (from datasheet)
- R_θCS = case-to-heatsink thermal resistance ≈ 0.5°C/W (with thermal pad)
- R_θSA = heatsink-to-ambient thermal resistance (depends on heatsink size)

**Without a heatsink:** R_θJA = 62°C/W (junction to ambient, free air)
- P_max = (175 - 25) / 62 = **2.4W** in free air — only about 11A!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Thermal calculation for IRFZ44N in different heatsink scenarios

R_thJC = 1.6    # °C/W junction to case
R_thCS = 0.5    # °C/W case to heatsink (with thermal pad)
T_A = 40        # °C ambient (warm environment)
T_J_max = 150   # °C design limit (leaving margin below 175°C absolute max)

# Heatsink scenarios
scenarios = [
    ('No heatsink (free air)', 62, 'red'),      # R_thJA = 62°C/W
    ('Small clip-on heatsink', 20, 'orange'),    # R_thSA ≈ 18°C/W
    ('Medium heatsink', 8, 'green'),             # R_thSA ≈ 6°C/W  
    ('Large heatsink + fan', 3, 'blue'),         # R_thSA ≈ 1°C/W
    ('Infinite heatsink (T_C=25°C)', 1.6, 'purple'),  # Only R_thJC
]

I_D = np.linspace(0, 50, 500)
R_DS_on_25C = 0.0175  # Ω at 25°C

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

for name, R_thJA, color in scenarios:
    # Need iterative calculation because R_DS(on) depends on temperature
    # which depends on power which depends on R_DS(on)...
    # Simplified: use R_DS(on) at 100°C as worst-case estimate
    R_DS_hot = R_DS_on_25C * (1 + 0.0065 * 75)  # ~75°C above 25°C average
    
    P_D = I_D**2 * R_DS_hot
    T_J = T_A + P_D * R_thJA
    
    ax1.plot(I_D, T_J, color=color, linewidth=2, label=name.split('(')[0].strip())
    
    # Find max current for this scenario
    P_max = (T_J_max - T_A) / R_thJA
    I_max = np.sqrt(P_max / R_DS_hot)
    ax2.barh(name.split('(')[0].strip(), min(I_max, 49), color=color, alpha=0.7)
    ax2.text(min(I_max, 49) + 0.5, name.split('(')[0].strip(), 
            f'{min(I_max, 49):.1f}A ({P_max:.1f}W)', fontsize=9, va='center')

ax1.axhline(y=T_J_max, color='red', linestyle='--', linewidth=2, alpha=0.7, label='T_J design limit (150°C)')
ax1.set_xlabel('Drain Current I_D (A)', fontsize=12)
ax1.set_ylabel('Junction Temperature T_J (°C)', fontsize=12)
ax1.set_title('IRFZ44N Junction Temperature vs Current\n(T_A = 40°C)', fontsize=13)
ax1.legend(fontsize=8, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 50)
ax1.set_ylim(0, 200)

ax2.set_xlabel('Maximum Continuous Current (A)', fontsize=12)
ax2.set_title('Max Current for Each Heatsink Scenario\n(T_J < 150°C, T_A = 40°C)', fontsize=13)
ax2.set_xlim(0, 55)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("The heatsink makes a MASSIVE difference!")
print("")
print("Without heatsink: ~3-4A max (in warm environment)")
print("With large heatsink: ~30A continuous")
print("The '49A' rating is only achievable with near-perfect cooling.")

## Worked Example: Can the IRFZ44N Switch 10A at 12V?

**Design scenario:** You want to switch a 12V, 10A DC load (e.g., LED strip, heater, motor) using the IRFZ44N with a small heatsink.

Let's work through the full calculation.

In [ ]:
import numpy as np

print("=" * 65)
print("IRFZ44N Design Check: Switching 10A at 12V")
print("=" * 65)

# Given
V_supply = 12.0     # V
I_load = 10.0       # A
V_GS = 10.0         # V gate drive
T_A = 40.0          # °C ambient
R_thSA = 10.0       # °C/W (small clip-on heatsink)
R_thJC = 1.6        # °C/W (from datasheet)
R_thCS = 0.5        # °C/W (with thermal pad)

print(f"\n--- Step 1: Check Voltage Rating ---")
V_DSS = 55  # V
print(f"V_supply = {V_supply}V, V_DSS(max) = {V_DSS}V")
print(f"Margin: {V_DSS - V_supply}V → {'PASS' if V_supply < V_DSS * 0.8 else 'MARGINAL'}")
print(f"(Rule of thumb: use < 80% of V_DSS = {V_DSS * 0.8}V)")

print(f"\n--- Step 2: Check Current Rating ---")
I_D_max = 49  # A at T_C = 25°C
print(f"I_load = {I_load}A, I_D(max) = {I_D_max}A")
print(f"{'PASS' if I_load < I_D_max else 'FAIL'} — but need to check thermal!")

print(f"\n--- Step 3: Calculate Power Dissipation ---")
R_DS_on_25C = 0.024  # Using MAX value (24 mΩ) for worst case!
print(f"R_DS(on) at 25°C = {R_DS_on_25C * 1000:.1f} mΩ (using DATASHEET MAX, not typical!)")

# Iterative thermal calculation
T_J = T_A  # Start with cold device
for iteration in range(10):
    # R_DS(on) at current temperature
    R_DS_on = R_DS_on_25C * (1 + 0.0065 * (T_J - 25))
    # Power dissipation
    P_D = I_load**2 * R_DS_on
    # Junction temperature
    R_thJA = R_thJC + R_thCS + R_thSA
    T_J_new = T_A + P_D * R_thJA
    if abs(T_J_new - T_J) < 0.01:
        break
    T_J = T_J_new

R_DS_on_hot = R_DS_on_25C * (1 + 0.0065 * (T_J - 25))
P_D_final = I_load**2 * R_DS_on_hot

print(f"\n--- Step 4: Thermal Calculation (iterative) ---")
print(f"R_θJA = R_θJC + R_θCS + R_θSA = {R_thJC} + {R_thCS} + {R_thSA} = {R_thJA}°C/W")
print(f"")
print(f"After convergence ({iteration + 1} iterations):")
print(f"  R_DS(on) at T_J = {R_DS_on_hot * 1000:.1f} mΩ")
print(f"  P_D = I_D² × R_DS(on) = {I_load}² × {R_DS_on_hot * 1000:.1f}mΩ = {P_D_final:.2f} W")
print(f"  T_J = T_A + P_D × R_θJA = {T_A} + {P_D_final:.2f} × {R_thJA} = {T_J:.1f}°C")

print(f"\n--- Step 5: Check Against Limits ---")
T_J_limit = 175
print(f"T_J = {T_J:.1f}°C vs T_J(max) = {T_J_limit}°C → {'PASS' if T_J < T_J_limit else 'FAIL'}")
print(f"T_J = {T_J:.1f}°C vs design target 125°C → {'PASS (good margin)' if T_J < 125 else 'MARGINAL' if T_J < 150 else 'FAIL'}")

print(f"\n--- Step 6: V_DS When ON ---")
V_DS_on = I_load * R_DS_on_hot
print(f"V_DS(on) = I_D × R_DS(on) = {I_load} × {R_DS_on_hot * 1000:.1f}mΩ = {V_DS_on * 1000:.0f} mV")
print(f"This is the voltage 'lost' across the switch — {V_DS_on / V_supply * 100:.1f}% of supply.")

print(f"\n--- Step 7: Gate Drive Check ---")
print(f"V_GS = {V_GS}V")
print(f"V_GS(th) max = 4.0V")
print(f"R_DS(on) spec'd at V_GS = 10V → {'PASS' if V_GS >= 10 else 'NEED MORE GATE DRIVE'}")

print(f"\n{'=' * 65}")
print(f"VERDICT: {'PASS — IRFZ44N can switch 10A at 12V with a small heatsink!' if T_J < 125 else 'NEEDS ATTENTION — consider larger heatsink'}")
print(f"{'=' * 65}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Design exercise: Select gate drive voltage to achieve target R_DS(on)

print("=" * 60)
print("Design Exercise: Gate Drive Voltage Selection")
print("=" * 60)
print()
print("Target: R_DS(on) < 25 mΩ for the IRFZ44N")
print("(This is the max datasheet spec at V_GS = 10V)")
print()

# R_DS(on) vs V_GS data (from datasheet approximation)
VGS_points = np.array([4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8.0, 10.0, 12.0, 15.0, 20.0])
RDS_points = np.array([200, 80, 45, 32, 26, 21, 19, 17.5, 16.5, 16.0, 15.5])  # mΩ

fig, ax = plt.subplots(figsize=(10, 6))

VGS_fine = np.linspace(4.0, 20.0, 300)
RDS_fine = np.interp(VGS_fine, VGS_points, RDS_points)

ax.plot(VGS_fine, RDS_fine, 'b-', linewidth=2.5, label='R_DS(on) vs V_GS')
ax.plot(VGS_points, RDS_points, 'bo', markersize=6)

# Target line
target_rds = 25  # mΩ
ax.axhline(y=target_rds, color='red', linestyle='--', linewidth=2, label=f'Target: {target_rds} mΩ')

# Find minimum V_GS to achieve target
# Interpolate to find where RDS crosses target
vgs_min = np.interp(target_rds, RDS_points[::-1], VGS_points[::-1])
ax.axvline(x=vgs_min, color='green', linestyle=':', linewidth=2)
ax.annotate(f'Minimum V_GS ≈ {vgs_min:.1f}V\nfor R_DS(on) < {target_rds} mΩ',
           xy=(vgs_min, target_rds), xytext=(vgs_min + 3, target_rds + 40),
           fontsize=12, color='green', fontweight='bold',
           arrowprops=dict(arrowstyle='->', color='green', lw=2),
           bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

# Show common drive voltage options
drive_options = [
    ('3.3V MCU', 3.3, 'TOO LOW!'),
    ('5V logic', 5.0, ''),
    ('12V gate driver', 12.0, ''),
]

for label, v, note in drive_options:
    if v >= VGS_points[0]:
        rds = np.interp(v, VGS_points, RDS_points)
        color = 'green' if rds <= target_rds else 'red'
        status = 'PASS' if rds <= target_rds else 'FAIL'
        ax.plot(v, rds, 's', markersize=10, color=color)
        text = f'{label}: {rds:.0f}mΩ [{status}]' + (f' {note}' if note else '')
        ax.annotate(text, xy=(v, rds), xytext=(v + 0.5, rds + 15),
                   fontsize=9, color=color)

ax.set_xlabel('V_GS (V)', fontsize=12)
ax.set_ylabel('R_DS(on) (mΩ)', fontsize=12)
ax.set_title('IRFZ44N: Selecting Gate Drive Voltage\nfor Target R_DS(on)', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(3, 21)
ax.set_ylim(0, 120)

plt.tight_layout()
plt.show()

print(f"To achieve R_DS(on) < {target_rds} mΩ, you need V_GS ≥ {vgs_min:.1f}V")
print(f"")
print(f"At 5V gate drive: R_DS(on) ≈ 45 mΩ — almost 3x the 10V specification!")
print(f"At 10V gate drive: R_DS(on) ≈ 17.5 mΩ — meets the datasheet spec.")
print(f"At 12V gate drive: R_DS(on) ≈ 16.5 mΩ — diminishing returns.")
print(f"")
print(f"CONCLUSION: The IRFZ44N requires a 10V+ gate driver. It is NOT logic-level!")
print(f"For 3.3V/5V direct drive, use a logic-level MOSFET like IRLZ44N (note the 'L').")

## Experiment — Verify Datasheet Parameters

Using the IRFZ44N on your bench, verify these datasheet values:

### 1. Measure V_GS(th)

```
    Bench PSU (0-10V, current limit 10mA)
         │
        [1kΩ]
         │
         ├──── Drain ─┐
         │             │ (short D to G for threshold test)
    Pot ─├──── Gate  ──┘
   0-5V  │
    DMM ──┤ (measure V_GS)
         │
         └──── Source ── GND
```

Slowly increase V_GS. When current begins to flow (voltage across 1kΩ appears), you've found V_GS(th).

**Expected: 2.0 - 4.0V** (your specific device will be one value in this range)

### 2. Measure R_DS(on)

Follow the procedure from Notebook 01, Experiment 3. Apply V_GS = 10V, pass a known current, measure V_DS.

**Expected: 17.5 mΩ typical, < 24 mΩ max**

## Simulation — Build It in Falstad

Explore MOSFET switching behavior and the parameters you see on a datasheet.

**What to build:**

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an N-channel MOSFET with a 100Ω load resistor from 12V to the drain
3. Connect the source to ground
4. Connect an adjustable DC voltage source to the gate

**What to observe:**
1. **Threshold behavior:** Increase V_GS from 0V — at the threshold voltage, the MOSFET begins conducting. Below threshold: V_DS ≈ V_DD (all voltage across the MOSFET). Above threshold: V_DS drops as the MOSFET turns on.
2. **R_DS(on):** With V_GS well above threshold (e.g. 10V), hover over the MOSFET — V_DS should be very small. The ratio V_DS / I_D is R_DS(on), the key datasheet parameter for power MOSFETs.
3. **No gate current:** Hover over the gate lead — essentially zero current, confirming voltage-controlled operation.

**What to try:**
- Record V_DS at V_GS = 4V, 5V, 7V, 10V — notice how R_DS(on) decreases as V_GS increases (this is why datasheets specify R_DS(on) at a particular V_GS)
- Increase the load current by decreasing the load resistance — at some point the MOSFET's voltage drop becomes significant, and it starts dissipating real power (P = I_D × V_DS)

## Datasheet Connection — Summary Checklist

When evaluating any MOSFET datasheet, check these in order:

### Quick Screening
- [ ] **V_DSS** ≥ your max voltage (with margin for spikes)
- [ ] **I_D** ≥ your max current (but check thermal!)
- [ ] **V_GS(th)** compatible with your gate drive voltage?
- [ ] **R_DS(on)** specified at your available V_GS?

### Power/Thermal Check
- [ ] **P_D** = I_D² × R_DS(on) at your operating current
- [ ] Use **R_DS(on) at max operating temperature** (not 25°C!)
- [ ] **T_J** = T_A + P_D × R_θJA — is it below 125°C design target?
- [ ] Is the heatsink adequate?

### Switching Check (if PWM/high frequency)
- [ ] **Q_g** — can your driver supply I = Q_g / t_switch?
- [ ] **Switching times** — fast enough for your frequency?
- [ ] **Gate drive power** = Q_g × V_GS × f_switch

### Gotchas
- [ ] Is the tab electrically connected to drain? (Need insulating pad for heatsink?)
- [ ] Logic level or standard? (Need gate driver?)
- [ ] Body diode recovery time acceptable? (For bridge circuits)

## Checkpoint Questions

1. **The IRFZ44N datasheet says I_D = 49A.** Can you actually run 49A through it on your breadboard? Why or why not?

2. **R_DS(on) is specified as 17.5 mΩ at V_GS = 10V.** You only have a 5V logic signal available. What's the problem, and what are two possible solutions?

3. **You're running the IRFZ44N at 10A in a 50°C ambient with no heatsink (R_θJA = 62°C/W).** Calculate T_J. Is this safe?

4. **Why does the datasheet specify R_DS(on) at a specific I_D (25A)?** Doesn't R_DS(on) stay constant regardless of current?

5. **The gate charge Q_g = 63 nC.** You want to switch at 500 kHz. Calculate: (a) Required gate driver current for 50 ns switching. (b) Gate drive power loss.

6. **You're bolting the IRFZ44N to a grounded aluminum heatsink.** Why might this be a problem? What's the solution?

7. **The body diode has V_SD = 1.3V and t_rr = 51 ns.** In what type of circuit would these parameters matter?

8. **Design problem:** You need to switch a 24V, 20A load. Check if the IRFZ44N is suitable. What V_GS will you use? What heatsink R_θSA is needed to keep T_J < 125°C at T_A = 40°C?

---

*Answers:*
1. *No! That rating assumes T_C = 25°C (massive heatsink). Without heatsink, R_θJA = 62°C/W limits you to ~2.4W → ~11A max. On a breadboard, even less.*
2. *At V_GS = 5V, R_DS(on) is ~45 mΩ (2.5x worse). Solutions: (a) Use a gate driver to boost to 10V, or (b) use a logic-level MOSFET (IRLZ44N, IRLB8721).*
3. *P_D = 10² × 0.024 (max) = 2.4W. T_J = 50 + 2.4 × 62 = 199°C. Way above 175°C limit — NOT safe! Need a heatsink.*
4. *R_DS(on) is slightly current-dependent due to self-heating and channel effects. The test condition defines the measurement point for consistency and comparison between devices.*
5. *(a) I_g = 63 nC / 50 ns = 1.26 A peak. (b) P_gate = 63 nC × 10V × 500 kHz = 315 mW.*
6. *The metal tab is the DRAIN — which carries high voltage! Bolting it to a grounded heatsink shorts the drain to ground. Use an insulating pad (silicone, mica, or Kapton) and insulating shoulder washer on the bolt.*
7. *In half-bridge or full-bridge circuits (H-bridges, motor drives) where the body diode conducts during dead time. The 1.3V forward drop causes losses, and the 51 ns reverse recovery can cause shoot-through current spikes.*
8. *V_DSS = 55V > 24V (OK, but only 31V margin — watch for inductive spikes!). At 20A: P_D = 20² × 0.024 = 9.6W (cold), maybe 15W hot. For T_J < 125°C: R_θJA = (125-40)/15 = 5.7°C/W. R_θSA = 5.7 - 1.6 - 0.5 = 3.6°C/W. Need a decent heatsink. V_GS = 10V minimum (use 12V gate driver).*